# Revisit all permutations sound paradigm with the new metrics
The last iteration of my experiments have used an stimulation paramete which I have called triplet
(three sound fragments coming from the same source) which enables the comparison between sounds with multiple
degrees of difference. This however limits the diversity of sounds used for stimulation. In the past
I have used four different sounds and consider all the permutations between them used as context and probes
This was somewhat replaced by the triplets approach, but keep recording some data with this paradigm.

It is time to analyse this data with the latest metrics for contextual effects: significant absolute sum
and significant absolute mass center


to do:
1. load DF, check general distribution of cells in paramter space


In [1]:
import itertools as itt
import pathlib as pl
from configparser import ConfigParser
from textwrap import fill

import joblib as jl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as sst
import seaborn as sns
from cycler import cycler
from ipywidgets import interact, fixed

import src.visualization.fancy_plots as fplt
from src.data.cache import set_name

[nems.configs.defaults INFO] Saving log messages to /tmp/nems/NEMS 2020-11-05 133345.log


In [ ]:
plt.style.use('dark_background')
color_cycler = cycler(color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
                            '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])
params = {'legend.fontsize': 'medium',
          'axes.labelsize': 15,
          'axes.titlesize': 20,
          'axes.spines.top': False,
          'axes.spines.right': False,
          'axes.prop_cycle': color_cycler,
          'xtick.labelsize': 11,
          'ytick.labelsize': 11,
          'lines.markersize': 8,
          'figure.titlesize': 30,
          'figure.figsize': [6,6],
          'figure.autolayout':True,
          'svg.fonttype': 'none',
          'font.sans-serif': 'Arial'
          }
plt.rcParams.update(params)